In [5]:
#i21-0381
#Question 1

import random
import time
import sys
from collections import deque

def generate_sudoku_board():
    board = [[0] * 9 for _ in range(9)]
    solve_sudoku(board)  #create board
    remove_cells(board, 45)  #remove 45 random cells
    return board

def remove_cells(board, num_cells_to_remove):
    cells = [(row, col) for row in range(9) for col in range(9)]
    random.shuffle(cells)
    for i in range(num_cells_to_remove):
        row, col = cells[i]
        board[row][col] = 0
        
#def fill_cells(board, num_cells_to_fill):
#    # Fill a specified number of cells in the Sudoku board
#    cells = [(row, col) for row in range(9) for col in range(9)]
#    random.shuffle(cells)
#    for i in range(num_cells_to_fill):
#        row, col = cells[i]
#        num = random.randint(1, 9)
#        while not is_valid_move(board, row, col, num):
#            num = random.randint(1, 9)
#        board[row][col] = num        

def is_valid_move(board, row, col, num):
    return (
        is_safe_row(board, row, num) and
        is_safe_col(board, col, num) and
        is_safe_box(board, row - row % 3, col - col % 3, num)
    )

def is_safe_row(board, row, num):
    return num not in board[row]

def is_safe_col(board, col, num):
     return all(board[i][col] != num for i in range(9))

def is_safe_box(board, start_row, start_col, num):
    
    return all(
        board[start_row + i][start_col + j] != num
        for i in range(3) for j in range(3)
    )

def solve_sudoku(board):
    start_time = time.time()    #for time cmplexity
    empty_row, empty_col = find_empty_cell(board) #for space complexity
    
    if empty_row is None:
        end_time = time.time()
        running_time = end_time - start_time
        total_space_complexity = sys.getsizeof(board)
        return True, running_time, total_space_complexity  #if the puzzle is already solved. to generate the initial board
    
    row, col = select_unassigned_variable(board)
    for value in order_domain_values(board, row, col):
        if is_valid_move(board, row, col, value):
            board[row][col] = value
            if solve_sudoku(board)[0]:
                end_time = time.time()
                running_time = end_time - start_time
                total_space_complexity = sys.getsizeof(board)
                return True, running_time, total_space_complexity
            board[row][col] = 0  #Backtrack
    end_time = time.time()
    running_time = end_time - start_time
    total_space_complexity = sys.getsizeof(board)
    return False, running_time, total_space_complexity

def find_empty_cell(board):
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                return row, col
    return None, None  #None if no empty cell found

def select_unassigned_variable(board):
    #select the variable with the fewest remaining values (MRV)
    min_values = float('inf')
    min_row, min_col = None, None
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                count = len(order_domain_values(board, row, col))
                if count < min_values:
                    min_values = count
                    min_row, min_col = row, col
    return min_row, min_col

def order_domain_values(board, row, col):
    #order the domain values based on the number of conflicts (Least Constraining Value + heuristic)
    values = list(range(1, 10))
    values.sort(key=lambda x: count_conflicts(board, row, col, x))
    return values

def count_conflicts(board, row, col, value):
    #number of conflicts (violations of Sudoku constraints) for a given value
    conflicts = 0
    for i in range(9):
        if board[row][i] == value or board[i][col] == value:
            conflicts += 1
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if board[start_row + i][start_col + j] == value:
                conflicts += 1
    return conflicts

def print_board(board):
    for i, row in enumerate(board):
        if i % 3 == 0 and i != 0:
            print("-" * 21)  
        for j, num in enumerate(row):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")  
            if num == 0:
                print("0", end=" ")
            else:
                print(num, end=" ")
        print()

if __name__ == "__main__":
    sudoku_board = generate_sudoku_board()
    print("Generated Sudoku puzzle:")
    print_board(sudoku_board)
    print("\nSolving Sudoku puzzle...\n")
    solved, time, space = solve_sudoku(sudoku_board)
    if solved:
        print("Solved Sudoku puzzle:")
        print_board(sudoku_board)
    else:
        print("No solution exists for the given Sudoku puzzle.")

    print()
    print("Running time:", time, "seconds")
    print("Total space complexity:", space, "bytes")


Generated Sudoku puzzle:
0 2 0 | 4 0 6 | 7 0 0 
4 5 0 | 0 0 9 | 0 0 3 
0 8 9 | 1 0 0 | 0 5 6 
---------------------
2 0 0 | 3 0 0 | 0 0 0 
0 0 0 | 8 0 7 | 2 1 0 
0 9 0 | 0 0 0 | 0 6 0 
---------------------
0 0 1 | 0 4 0 | 0 7 8 
6 0 2 | 9 0 8 | 0 3 1 
9 0 8 | 0 0 1 | 6 0 2 

Solving Sudoku puzzle...

Solved Sudoku puzzle:
1 2 3 | 4 5 6 | 7 8 9 
4 5 6 | 7 8 9 | 1 2 3 
7 8 9 | 1 2 3 | 4 5 6 
---------------------
2 1 4 | 3 6 5 | 8 9 7 
3 6 5 | 8 9 7 | 2 1 4 
8 9 7 | 2 1 4 | 3 6 5 
---------------------
5 3 1 | 6 4 2 | 9 7 8 
6 4 2 | 9 7 8 | 5 3 1 
9 7 8 | 5 3 1 | 6 4 2 

Running time: 0.03183174133300781 seconds
Total space complexity: 184 bytes


In [6]:
#Question 2

import numpy as np
import random
import time
import sys

N = 3  # Magic Square size
MAGIC_SUM = 15
POPULATION_SIZE = 100
GENERATIONS = 2000
MUTATION_RATE = 0.3

def generate_individual():
    return np.random.permutation(np.arange(1, N**2 + 1)).reshape(N, N)

def fitness(state):
    fitness_val = 0
    for i in range(N):
        fitness_val += abs(np.sum(state[i, :]) - MAGIC_SUM)
        fitness_val += abs(np.sum(state[:, i]) - MAGIC_SUM)
    fitness_val += abs(np.sum(np.diag(state)) - MAGIC_SUM)
    fitness_val += abs(np.sum(np.diag(np.fliplr(state))) - MAGIC_SUM)

    return fitness_val

def mutate(state):
    new_state = state.flatten()
    swap_indices = np.random.choice(np.arange(N**2), 2, replace=False)
    new_state[swap_indices[0]], new_state[swap_indices[1]] = new_state[swap_indices[1]], new_state[swap_indices[0]]
    return new_state.reshape(N, N)

def generate_initial_population():
    population = []
    for _ in range(POPULATION_SIZE):
        state = generate_individual()
        population.append(state)
        
    return population

def genetic_algorithm():
    population = generate_initial_population()
    best_fitness = float('inf')
    best_state = None
    min_fitness_solution = None

    # Calculate space complexity for the population list
    population_size_bytes = sys.getsizeof(population)

    start_time = time.time()

    for generation in range(GENERATIONS):
        fitnesses = [fitness(state) for state in population]

        best_fitness_index = np.argmin(fitnesses)
        if fitnesses[best_fitness_index] < best_fitness:
            best_fitness = fitnesses[best_fitness_index]
            best_state = population[best_fitness_index]

        if best_fitness == 0:  # Found a magic square
            min_fitness_solution = best_state
            break

        # Update minimum fitness solution
        min_fitness_index = np.argmin(fitnesses)
        if min_fitness_solution is None or fitnesses[min_fitness_index] < fitness(min_fitness_solution):
            min_fitness_solution = population[min_fitness_index]

        # Create a new generation through mutation
        new_population = []
        for state in population:
            if random.random() < MUTATION_RATE:
                mutated_state = mutate(state)
                new_population.append(mutated_state)
            else:
                new_population.append(state)
        
        # Calculate space complexity for the new_population list
        new_population_size_bytes = sys.getsizeof(new_population)
        
        population = new_population

    end_time = time.time()
    running_time = end_time - start_time

    # Calculate total space complexity
    total_space_complexity = population_size_bytes + new_population_size_bytes

    return best_state, best_fitness, min_fitness_solution, running_time, total_space_complexity

if __name__ == "__main__":
    best_state, best_fitness, min_fitness_solution, running_time, total_space_complexity = genetic_algorithm()
    if min_fitness_solution is not None:
        print("Minimum fitness solution found:")
        print(min_fitness_solution)
        print("Fitness:", best_fitness)
    else:
        print("No minimum fitness solution found.")

    print("Running time:", running_time, "seconds")
    print("Total space complexity:", total_space_complexity, "bytes")


Minimum fitness solution found:
[[4 9 2]
 [3 5 7]
 [8 1 6]]
Fitness: 0
Running time: 0.1532151699066162 seconds
Total space complexity: 1840 bytes
